# Colour 0.3.8 - Computational Changes

[colour-science.org](http://colour-science.org) - May 15, 2016

> Note: This document is not final.

# Spectral Computations

This section illustrates some of the computational differences to expect when using the new spectral code of *Colour 0.3.8* which is compliant with **ASTM E308–15**.

In the following example, the overall $\Delta E_{2000}$ colour difference mean between *Colour 0.3.7* and *Colour 0.3.8* is **0.005** (0.01 standard deviation).

In [1]:
import numpy as np
import pandas

import colour

import warnings

warnings.filterwarnings('ignore')

delta_E = colour.delta_E_CIE2000


def spectral_to_XYZ(spd,
                    cmfs=colour.STANDARD_OBSERVERS_CMFS.get(
                        'CIE 1931 2 Degree Standard Observer'),
                    illuminant=None):
    shape = cmfs.shape
    if spd.shape != cmfs.shape:
        spd = spd.clone().zeros(shape)

    if illuminant is None:
        illuminant = colour.ones_spd(shape)
    else:
        if illuminant.shape != cmfs.shape:
            illuminant = illuminant.clone().zeros(shape)

    spd = spd.values
    x_bar, y_bar, z_bar = (cmfs.x_bar.values,
                           cmfs.y_bar.values,
                           cmfs.z_bar.values)
    illuminant = illuminant.values

    x_products = spd * x_bar * illuminant
    y_products = spd * y_bar * illuminant
    z_products = spd * z_bar * illuminant

    normalising_factor = 100 / np.sum(y_bar * illuminant)

    XYZ = np.array([normalising_factor * np.sum(x_products),
                    normalising_factor * np.sum(y_products),
                    normalising_factor * np.sum(z_products)])

    return XYZ


XYZ_0_3_7 = []
XYZ_0_3_8 = []
for spd in colour.ILLUMINANTS_RELATIVE_SPDS.values():
    XYZ_0_3_7.append(spectral_to_XYZ(spd) * spd.shape.interval / 100)
    XYZ_0_3_8.append(colour.spectral_to_XYZ(spd) / 100)

d_E = delta_E(
    colour.XYZ_to_Lab(XYZ_0_3_7),
    colour.XYZ_to_Lab(XYZ_0_3_8))

pandas.DataFrame({'Overall - ΔE': d_E}).describe()

,Overall - ΔE
count,41.000000
mean,0.005400
std,0.014937
min,0.000011
25%,0.001047
50%,0.002432
75%,0.004388
max,0.096603


## Colourspace Models

This section illustrates colour differences occuring as a result of *D50* and *D65* illuminants chromaticity coordinates changes (CIE 1931 2° Standard Observer) and *Adobe RGB 1998*, *Prophoto RGB*, and *sRGB* colourspaces normalised primary matrices updates. 

Those alterations are the result of [a decision to better comply](https://github.com/colour-science/colour/issues/220) with the cited RGB colourspaces specifications (even if it means using a rounded less precise dataset).

> Note: Almost all the RGB colourspaces and various other objects defined in *Colour* are using the *D50* or *D65* illuminants chromaticity coordinates, thus some minor numerical differences between *Colour 0.3.7* and *Colour 0.3.8* are expected for those objects.

RGB values of a X-Rite ColorChecker 2005 are computed in various *RGB* colourspace models using *Colour 0.3.7* and *Colour 0.3.8* dataset. Colour differences of the output values are taken: the overall $\Delta E_{2000}$ colour difference mean is **0.006** (0.002 standard deviation) and reaches **0.03** (0.01 standard deviation) with high dynamic range values (Luminance → 10000).

*CIE Lab* conversions only accounting for illuminants chromaticity coordinates changes have an overall $\Delta E_{2000}$ colour difference mean of **0.006** (0.003 standard deviation) and reaches **0.03** (0.03 standard deviation) with high dynamic range values.

### RGB Colourspaces

In [2]:
name, data, illuminant = colour.COLOURCHECKERS['ColorChecker 2005']
XYZ = np.asarray([colour.xyY_to_XYZ(np.array([x, y, Y]))
                  for index, name, x, y, Y in data])
# X, Y, Z = colour.tsplit(XYZ)
# XYZ_HDR = colour.tstack((X, Y * 10000, Z))
XYZ_HDR = XYZ * 10000

D50_0_3_7 = np.array([0.34567, 0.35850])
D50_0_3_8 = np.array([0.34570, 0.35850])
D65_0_3_7 = np.array([0.31271, 0.32902])
D65_0_3_8 = np.array([0.31270, 0.32900])

sRGB_XYZ_TO_RGB_MATRIX_0_3_7 = np.linalg.inv(np.array(
    [[0.41238656, 0.35759149, 0.18045049],
     [0.21263682, 0.71518298, 0.0721802],
     [0.01933062, 0.11919716, 0.95037259]]))

sRGB_XYZ_TO_RGB_MATRIX_0_3_8 = np.linalg.inv(np.array(
    [[0.4124, 0.3576, 0.1805],
     [0.2126, 0.7152, 0.0722],
     [0.0193, 0.1192, 0.9505]]))

PROPHOTO_XYZ_to_RGB_MATRIX_0_3_7 = np.linalg.inv(np.array(
    [[7.97667235e-01, 1.35192231e-01, 3.13525290e-02],
     [2.88037454e-01, 7.11876883e-01, 8.56626476e-05],
     [0.00000000e+00, 0.00000000e+00, 8.25188285e-01]]))

PROPHOTO_XYZ_to_RGB_MATRIX_0_3_8 = np.linalg.inv(np.array(
    [[0.7977, 0.1352, 0.0313],
     [0.2880, 0.7119, 0.0001],
     [0.0000, 0.0000, 0.8249]]))

ADOBE_RGB_1998_XYZ_to_RGB_MATRIX_0_3_7 = np.linalg.inv(np.array(
    [[0.57666809, 0.18556195, 0.1881985],
     [0.29734449, 0.62737611, 0.0752794],
     [0.02703132, 0.07069027, 0.99117879]]))

ADOBE_RGB_1998_XYZ_to_RGB_MATRIX_0_3_8 = np.linalg.inv(np.array(
    [[0.57667, 0.18556, 0.18823],
     [0.29734, 0.62736, 0.07529],
     [0.02703, 0.07069, 0.99134]]))

RGB_to_XYZ = lambda x: colour.RGB_to_XYZ(
    x,
    D65_0_3_8,
    D65_0_3_8,
    np.linalg.inv(sRGB_XYZ_TO_RGB_MATRIX_0_3_8))

# sRGB 1 - ΔE
RGB_0_3_7__1 = colour.XYZ_to_RGB(
    XYZ,
    D65_0_3_7,
    D65_0_3_7,
    sRGB_XYZ_TO_RGB_MATRIX_0_3_7)

RGB_0_3_8__1 = colour.XYZ_to_RGB(
    XYZ,
    D65_0_3_8,
    D65_0_3_8,
    sRGB_XYZ_TO_RGB_MATRIX_0_3_8)

d_E__1 = delta_E(
    colour.XYZ_to_Lab(RGB_to_XYZ(RGB_0_3_7__1)),
    colour.XYZ_to_Lab(RGB_to_XYZ(RGB_0_3_8__1)))

# sRGB 2 - ΔE
RGB_0_3_7__2 = colour.XYZ_to_RGB(
    XYZ,
    D50_0_3_7,
    D65_0_3_7,
    sRGB_XYZ_TO_RGB_MATRIX_0_3_7)

RGB_0_3_8__2 = colour.XYZ_to_RGB(
    XYZ,
    D50_0_3_8,
    D65_0_3_8,
    sRGB_XYZ_TO_RGB_MATRIX_0_3_8)

d_E__2 = delta_E(
    colour.XYZ_to_Lab(RGB_to_XYZ(RGB_0_3_7__2)),
    colour.XYZ_to_Lab(RGB_to_XYZ(RGB_0_3_8__2)))

# Prophoto 1 - ΔE
RGB_0_3_7__3 = colour.XYZ_to_RGB(
    XYZ,
    D50_0_3_7,
    D50_0_3_7,
    PROPHOTO_XYZ_to_RGB_MATRIX_0_3_7)

RGB_0_3_8__3 = colour.XYZ_to_RGB(
    XYZ,
    D50_0_3_8,
    D50_0_3_8,
    PROPHOTO_XYZ_to_RGB_MATRIX_0_3_8)

d_E__3 = delta_E(
    colour.XYZ_to_Lab(RGB_to_XYZ(RGB_0_3_7__3)),
    colour.XYZ_to_Lab(RGB_to_XYZ(RGB_0_3_8__3)))

# Prophoto 2 - ΔE
RGB_0_3_7__4 = colour.XYZ_to_RGB(
    XYZ,
    D65_0_3_7,
    D50_0_3_7,
    PROPHOTO_XYZ_to_RGB_MATRIX_0_3_7)

RGB_0_3_8__4 = colour.XYZ_to_RGB(
    XYZ,
    D65_0_3_8,
    D50_0_3_8,
    PROPHOTO_XYZ_to_RGB_MATRIX_0_3_8)

d_E__4 = delta_E(
    colour.XYZ_to_Lab(RGB_to_XYZ(RGB_0_3_7__4)),
    colour.XYZ_to_Lab(RGB_to_XYZ(RGB_0_3_8__4)))

# Adobe RGB (1998) 1 - ΔE
RGB_0_3_7__5 = colour.XYZ_to_RGB(
    XYZ,
    D65_0_3_7,
    D65_0_3_7,
    ADOBE_RGB_1998_XYZ_to_RGB_MATRIX_0_3_7)

RGB_0_3_8__5 = colour.XYZ_to_RGB(
    XYZ,
    D65_0_3_8,
    D65_0_3_8,
    ADOBE_RGB_1998_XYZ_to_RGB_MATRIX_0_3_8)

d_E__5 = delta_E(
    colour.XYZ_to_Lab(RGB_to_XYZ(RGB_0_3_7__5)),
    colour.XYZ_to_Lab(RGB_to_XYZ(RGB_0_3_8__5)))

# Adobe RGB (1998) 2 - ΔE
RGB_0_3_7__6 = colour.XYZ_to_RGB(
    XYZ,
    D50_0_3_7,
    D65_0_3_7,
    ADOBE_RGB_1998_XYZ_to_RGB_MATRIX_0_3_7)

RGB_0_3_8__6 = colour.XYZ_to_RGB(
    XYZ,
    D50_0_3_8,
    D65_0_3_8,
    ADOBE_RGB_1998_XYZ_to_RGB_MATRIX_0_3_8)

d_E__6 = delta_E(
    colour.XYZ_to_Lab(RGB_to_XYZ(RGB_0_3_7__6)),
    colour.XYZ_to_Lab(RGB_to_XYZ(RGB_0_3_8__6)))

print(
    """
    * Adobe RGB (1998) 1 - ΔE: Conversion to Adobe RGB (1998) v0.3.7 vs v0.3.8, no chromatic adaptation
    * Adobe RGB (1998) 2 - ΔE: Conversion to Adobe RGB (1998) v0.3.7 vs v0.3.8, chromatic adaptation from D65 to D50
    * ProPhoto RGB 1 - ΔE: Conversion to ProPhoto RGB v0.3.7 vs v0.3.8, no chromatic adaptation
    * ProPhoto RGB 2 - ΔE: Conversion to ProPhoto RGB v0.3.7 vs v0.3.8, chromatic adaptation from D65 to D50
    * sRGB 1 - ΔE: Conversion to sRGB v0.3.7 vs v0.3.8, no chromatic adaptation
    * sRGB 2 - ΔE: Conversion to sRGB v0.3.7 vs v0.3.8, chromatic adaptation from D50 to D65
    """)

results = {'sRGB 1 - ΔE': d_E__1,
           'sRGB 2 - ΔE': d_E__2,
           'ProPhoto RGB 1 - ΔE': d_E__3,
           'ProPhoto RGB 2 - ΔE': d_E__4,
           'Adobe RGB (1998) 1 - ΔE': d_E__5,
           'Adobe RGB (1998) 2 - ΔE': d_E__6}
results_dataframe = pandas.DataFrame(results)
results_dataframe.describe()


    * Adobe RGB (1998) 1 - ΔE: Conversion to Adobe RGB (1998) v0.3.7 vs v0.3.8, no chromatic adaptation
    * Adobe RGB (1998) 2 - ΔE: Conversion to Adobe RGB (1998) v0.3.7 vs v0.3.8, chromatic adaptation from D65 to D50
    * ProPhoto RGB 1 - ΔE: Conversion to ProPhoto RGB v0.3.7 vs v0.3.8, no chromatic adaptation
    * ProPhoto RGB 2 - ΔE: Conversion to ProPhoto RGB v0.3.7 vs v0.3.8, chromatic adaptation from D65 to D50
    * sRGB 1 - ΔE: Conversion to sRGB v0.3.7 vs v0.3.8, no chromatic adaptation
    * sRGB 2 - ΔE: Conversion to sRGB v0.3.7 vs v0.3.8, chromatic adaptation from D50 to D65
    


,Adobe RGB (1998) 1 - ΔE,Adobe RGB (1998) 2 - ΔE,ProPhoto RGB 1 - ΔE,ProPhoto RGB 2 - ΔE,sRGB 1 - ΔE,sRGB 2 - ΔE
count,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000
mean,0.004170,0.005795,0.006563,0.003545,0.006663,0.010531
std,0.002809,0.002609,0.003042,0.002023,0.004106,0.004785
min,0.001299,0.002380,0.003275,0.001292,0.000892,0.003563
25%,0.001998,0.003640,0.004191,0.001911,0.003877,0.007242
50%,0.003279,0.005360,0.005582,0.002900,0.005870,0.009160
75%,0.005507,0.007173,0.008309,0.005370,0.007110,0.012594
max,0.011429,0.012182,0.012422,0.008152,0.017947,0.022735


In [3]:
pandas.DataFrame({'Overall - ΔE': np.squeeze(
            results_dataframe.describe()[1:2].values)}).describe()

,Overall - ΔE
count,6.000000
mean,0.006211
std,0.002467
min,0.003545
25%,0.004576
50%,0.006179
75%,0.006638
max,0.010531


In [4]:
# sRGB 1 - HDR - ΔE
RGB_0_3_7__1 = colour.XYZ_to_RGB(
    XYZ_HDR,
    D65_0_3_7,
    D65_0_3_7,
    sRGB_XYZ_TO_RGB_MATRIX_0_3_7)

RGB_0_3_8__1 = colour.XYZ_to_RGB(
    XYZ_HDR,
    D65_0_3_8,
    D65_0_3_8,
    sRGB_XYZ_TO_RGB_MATRIX_0_3_8)

d_E__1 = delta_E(
    colour.XYZ_to_Lab(RGB_to_XYZ(RGB_0_3_7__1)),
    colour.XYZ_to_Lab(RGB_to_XYZ(RGB_0_3_8__1)))

# sRGB 2 - HDR - ΔE
RGB_0_3_7__2 = colour.XYZ_to_RGB(
    XYZ_HDR,
    D50_0_3_7,
    D65_0_3_7,
    sRGB_XYZ_TO_RGB_MATRIX_0_3_7)

RGB_0_3_8__2 = colour.XYZ_to_RGB(
    XYZ_HDR,
    D50_0_3_8,
    D65_0_3_8,
    sRGB_XYZ_TO_RGB_MATRIX_0_3_8)

d_E__2 = delta_E(
    colour.XYZ_to_Lab(RGB_to_XYZ(RGB_0_3_7__2)),
    colour.XYZ_to_Lab(RGB_to_XYZ(RGB_0_3_8__2)))

# Prophoto 1 - HDR - ΔE
RGB_0_3_7__3 = colour.XYZ_to_RGB(
    XYZ_HDR,
    D50_0_3_7,
    D50_0_3_7,
    PROPHOTO_XYZ_to_RGB_MATRIX_0_3_7)

RGB_0_3_8__3 = colour.XYZ_to_RGB(
    XYZ_HDR,
    D50_0_3_8,
    D50_0_3_8,
    PROPHOTO_XYZ_to_RGB_MATRIX_0_3_8)

d_E__3 = delta_E(
    colour.XYZ_to_Lab(RGB_to_XYZ(RGB_0_3_7__3)),
    colour.XYZ_to_Lab(RGB_to_XYZ(RGB_0_3_8__3)))

# Prophoto 2 - HDR - ΔE
RGB_0_3_7__4 = colour.XYZ_to_RGB(
    XYZ_HDR,
    D65_0_3_7,
    D50_0_3_7,
    PROPHOTO_XYZ_to_RGB_MATRIX_0_3_7)

RGB_0_3_8__4 = colour.XYZ_to_RGB(
    XYZ_HDR,
    D65_0_3_8,
    D50_0_3_8,
    PROPHOTO_XYZ_to_RGB_MATRIX_0_3_8)

d_E__4 = delta_E(
    colour.XYZ_to_Lab(RGB_to_XYZ(RGB_0_3_7__4)),
    colour.XYZ_to_Lab(RGB_to_XYZ(RGB_0_3_8__4)))

# Adobe RGB (1998) 1 - HDR - ΔE
RGB_0_3_7__5 = colour.XYZ_to_RGB(
    XYZ_HDR,
    D65_0_3_7,
    D65_0_3_7,
    ADOBE_RGB_1998_XYZ_to_RGB_MATRIX_0_3_7)

RGB_0_3_8__5 = colour.XYZ_to_RGB(
    XYZ_HDR,
    D65_0_3_8,
    D65_0_3_8,
    ADOBE_RGB_1998_XYZ_to_RGB_MATRIX_0_3_8)

d_E__5 = delta_E(
    colour.XYZ_to_Lab(RGB_to_XYZ(RGB_0_3_7__5)),
    colour.XYZ_to_Lab(RGB_to_XYZ(RGB_0_3_8__5)))

# Adobe RGB (1998) 2 - HDR - ΔE
RGB_0_3_7__6 = colour.XYZ_to_RGB(
    XYZ_HDR,
    D50_0_3_7,
    D65_0_3_7,
    ADOBE_RGB_1998_XYZ_to_RGB_MATRIX_0_3_7)

RGB_0_3_8__6 = colour.XYZ_to_RGB(
    XYZ_HDR,
    D50_0_3_8,
    D65_0_3_8,
    ADOBE_RGB_1998_XYZ_to_RGB_MATRIX_0_3_8)

d_E__6 = delta_E(
    colour.XYZ_to_Lab(RGB_to_XYZ(RGB_0_3_7__6)),
    colour.XYZ_to_Lab(RGB_to_XYZ(RGB_0_3_8__6)))

print(
    """
    * Adobe RGB (1998) 1 - HDR - ΔE: Conversion to Adobe RGB (1998) v0.3.7 vs v0.3.8, no chromatic adaptation
    * Adobe RGB (1998) 2 - HDR - ΔE: Conversion to Adobe RGB (1998) v0.3.7 vs v0.3.8, chromatic adaptation from D65 to D50
    * ProPhoto RGB 1 - HDR - ΔE: Conversion to ProPhoto RGB v0.3.7 vs v0.3.8, no chromatic adaptation
    * ProPhoto RGB 2 - HDR - ΔE: Conversion to ProPhoto RGB v0.3.7 vs v0.3.8, chromatic adaptation from D65 to D50
    * sRGB 1 - HDR - ΔE: Conversion to sRGB v0.3.7 vs v0.3.8, no chromatic adaptation
    * sRGB 2 - HDR - ΔE: Conversion to sRGB v0.3.7 vs v0.3.8, chromatic adaptation from D50 to D65
    """)

results = {'sRGB 1 - HDR - ΔE': d_E__1,
           'sRGB 2 - HDR - ΔE': d_E__2,
           'ProPhoto RGB 1 - HDR - ΔE': d_E__3,
           'ProPhoto RGB 2 - HDR - ΔE': d_E__4,
           'Adobe RGB (1998) 1 - HDR - ΔE': d_E__5,
           'Adobe RGB (1998) 2 - HDR - ΔE': d_E__6}
results_dataframe = pandas.DataFrame(results)
results_dataframe.describe()


    * Adobe RGB (1998) 1 - HDR - ΔE: Conversion to Adobe RGB (1998) v0.3.7 vs v0.3.8, no chromatic adaptation
    * Adobe RGB (1998) 2 - HDR - ΔE: Conversion to Adobe RGB (1998) v0.3.7 vs v0.3.8, chromatic adaptation from D65 to D50
    * ProPhoto RGB 1 - HDR - ΔE: Conversion to ProPhoto RGB v0.3.7 vs v0.3.8, no chromatic adaptation
    * ProPhoto RGB 2 - HDR - ΔE: Conversion to ProPhoto RGB v0.3.7 vs v0.3.8, chromatic adaptation from D65 to D50
    * sRGB 1 - HDR - ΔE: Conversion to sRGB v0.3.7 vs v0.3.8, no chromatic adaptation
    * sRGB 2 - HDR - ΔE: Conversion to sRGB v0.3.7 vs v0.3.8, chromatic adaptation from D50 to D65
    


,Adobe RGB (1998) 1 - HDR - ΔE,Adobe RGB (1998) 2 - HDR - ΔE,ProPhoto RGB 1 - HDR - ΔE,ProPhoto RGB 2 - HDR - ΔE,sRGB 1 - HDR - ΔE,sRGB 2 - HDR - ΔE
count,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000
mean,0.026184,0.017442,0.020993,0.015517,0.048194,0.030479
std,0.035712,0.010542,0.024208,0.013827,0.067167,0.020316
min,0.001960,0.004145,0.007118,0.001833,0.001322,0.005232
25%,0.005259,0.008038,0.010060,0.003494,0.008221,0.015646
50%,0.007359,0.014469,0.014516,0.012635,0.014617,0.023240
75%,0.029524,0.024952,0.018783,0.020148,0.044347,0.049502
max,0.132952,0.037742,0.120340,0.048273,0.203857,0.070433


In [5]:
pandas.DataFrame({'Overall - HDR - ΔE': np.squeeze(
            results_dataframe.describe()[1:2].values)}).describe()

,Overall - HDR - ΔE
count,6.000000
mean,0.026468
std,0.011996
min,0.015517
25%,0.018330
50%,0.023588
75%,0.029405
max,0.048194


### CIE Lab

In [6]:
d_E__1 = delta_E(
    colour.XYZ_to_Lab(XYZ, D50_0_3_7),
    colour.XYZ_to_Lab(XYZ, D50_0_3_8))

d_E__2 = delta_E(
    colour.XYZ_to_Lab(XYZ, D65_0_3_7),
    colour.XYZ_to_Lab(XYZ, D65_0_3_8))

print(
    """
    * CIE Lab 1 - ΔE: D50
    * CIE Lab 1 - ΔE: D65
    """)

results = {'CIE Lab 1 - ΔE': d_E__1,
           'CIE Lab 2 - ΔE': d_E__2}
results_dataframe = pandas.DataFrame(results)
results_dataframe.describe()


    * CIE Lab 1 - ΔE: D50
    * CIE Lab 1 - ΔE: D65
    


,CIE Lab 1 - ΔE,CIE Lab 2 - ΔE
count,24.000000,24.000000
mean,0.007691,0.003701
std,0.004800,0.001690
min,0.002643,0.001431
25%,0.005097,0.002529
50%,0.006238,0.003214
75%,0.008135,0.004475
max,0.021485,0.007566


In [7]:
pandas.DataFrame({'Overall - ΔE': np.squeeze(
            results_dataframe.describe()[1:2].values)}).describe()

,Overall - ΔE
count,2.000000
mean,0.005696
std,0.002821
min,0.003701
25%,0.004699
50%,0.005696
75%,0.006694
max,0.007691


In [8]:
d_E__1 = delta_E(
    colour.XYZ_to_Lab(XYZ_HDR, D50_0_3_7),
    colour.XYZ_to_Lab(XYZ_HDR, D50_0_3_8))

d_E__2 = delta_E(
    colour.XYZ_to_Lab(XYZ_HDR, D65_0_3_7),
    colour.XYZ_to_Lab(XYZ_HDR, D65_0_3_8))

print(
    """
    * CIE Lab 1 - HDR - ΔE: D50
    * CIE Lab 1 - HDR - ΔE: D65
    """)

results = {'CIE Lab 1 - ΔE': d_E__1,
           'CIE Lab 2 - ΔE': d_E__2}
results_dataframe = pandas.DataFrame(results)
results_dataframe.describe()


    * CIE Lab 1 - HDR - ΔE: D50
    * CIE Lab 1 - HDR - ΔE: D65
    


,CIE Lab 1 - ΔE,CIE Lab 2 - ΔE
count,24.000000,24.000000
mean,0.054747,0.012313
std,0.076240,0.009439
min,0.004040,0.002047
25%,0.011649,0.005269
50%,0.015807,0.007352
75%,0.065657,0.018146
max,0.257200,0.028689


In [9]:
pandas.DataFrame({'Overall - HDR - ΔE': np.squeeze(
            results_dataframe.describe()[1:2].values)}).describe()

,Overall - HDR - ΔE
count,2.000000
mean,0.033530
std,0.030005
min,0.012313
25%,0.022922
50%,0.033530
75%,0.044138
max,0.054747
